In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import ecopy as ep
from scipy.spatial.distance import pdist,squareform
from skbio.stats.ordination import pcoa
from scipy.interpolate import griddata
from skbio import DistanceMatrix
from skbio import diversity
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO,Phylo
from IPython.display import IFrame
import pickle

import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
import warnings
warnings.filterwarnings("ignore")

from function_diversity_analysis import Beta_Diversity_Contour_plot,Diversity_bar_plot,export_values

%matplotlib inline

In [4]:
# Example - 16S bar plots

# Load in alpha diversity data 
df_alpha_meta = pd.read_pickle('./results/df_alpha_meta.pkl')

with open("./results/dict_beta.pkl", "rb") as f:
     dict_beta = pickle.load(f)

In [5]:
AxB_table = [['season','year'],['year','season'],['year','region'],['region','season']]

In [6]:
df_alpha_meta['year'] = df_alpha_meta['year'].astype(str)

In [7]:
# 16S alpha/beta diversity reuslts - below will generate an interactive bar plot

Method_choice =  'observed_otus'
image_size = [800,500]

fig,layout = Diversity_bar_plot(df_alpha_meta = df_alpha_meta,AxB_table = AxB_table,div_a_col=Method_choice, diversity_type='alpha', cols=1,Color_pattern = 'Spectral')
fig['layout'].update(height=image_size[0], width=image_size[1],showlegend=False)
export_values(fig).to_csv('results/'+Method_choice+'.csv')
py.plot(fig,filename='results/'+Method_choice+'.html', auto_open=False)
IFrame(src='results/'+Method_choice+'.html',height=image_size[0]+50, width=image_size[1]+50)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]



In [8]:
# 16S alpha/beta diversity reuslts - below will generate an interactive pcoa plot
Factors = ['Temp', 'Sal', 'Density', 'O2', 'PO4', 'SiO3', 'NO3', 'NH3', 'Chla', 'PhaePig','ChlaAnom','PO4Anom', 'SiO3Anom', 'NO2Anom', 'NO3Anom','NCDepth', 'TempAnom']

In [9]:
Alpha_method_choice =  'observed_otus'
Beta_method_choice =  'unifrac_weighted'

In [12]:
# Initilize parameters
sym_index = df_alpha_meta.sort_index().index
df_alpha_meta = df_alpha_meta.loc[sym_index]
df_matrix = dict_beta[Beta_method_choice][sym_index].loc[sym_index]
a_div_col = Alpha_method_choice
group_factors = ['region','season','year']
contour_factors = Factors
node_scale = 10/df_alpha_meta[Alpha_method_choice].mean()
contour_resolution = 50
opacity_node = 0.9
opacity_contour = 0.4

# PCOA analysis
dm = DistanceMatrix(df_matrix.values.tolist(),df_matrix.index.tolist())
df_pcoa = pcoa(dm)
df_PCoA_Matrix = df_pcoa.samples

df_alpha_meta['x'] = df_PCoA_Matrix['PC1']
df_alpha_meta['y'] = df_PCoA_Matrix['PC2']

# PCoA plots

fig,layout = Beta_Diversity_Contour_plot(df_alpha_meta,df_PCoA_Matrix,a_div_col,group_factors,contour_factors,node_scale,contour_resolution,opacity_node,opacity_contour,contour_cmap='heatmap')
fig['layout'].update(height=600, width=1000,showlegend=True)
py.plot(fig,filename='results/PCoA.html', auto_open=False)
IFrame(src='results/PCoA.html',height=650, width=1050)